***
## notebook config

In [1]:
ON_COLAB = False

In [2]:
if ON_COLAB:
    from google.colab import drive
    from google.colab import files
    drive.mount('/content/drive', force_remount=True)
    
    !pip install --upgrade kaggle > /dev/null 2>&1
    !mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
    
    !free -h
    
    !pip install --upgrade category_encoders > /dev/null 2>&1
    !pip install --upgrade tsforest > /dev/null 2>&1
    !pip install --upgrade lightgbm > /dev/null 2>&1
    !pip install --upgrade optuna > /dev/null 2>&1

In [3]:
if ON_COLAB:
    !kaggle datasets download -d mavillan/meli-2021 --force --unzip
    !ls -halt
    input_path = "./"
    print("input_path:", input_path)
else:
    input_path = "../data"
    print("input_path:", input_path)

input_path: ../data


***

In [4]:
from copy import deepcopy
import gc
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate
from scipy import stats
import time
import yaml

import category_encoders as ce
import lightgbm as lgb
from tqdm import tqdm
from tsforest.forecast import LightGBMForecaster
import optuna

pd.set_option('display.max_columns', None)

In [5]:
def reduce_mem_usage(df, verbose=False):
    """
    Utility function to reduce the memory usage of pandas dataframes
    
    Parameters
    ----------
    df: pandas.Dataframe
    verbose: Boolean
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
class RMSSE():
    def __init__(self, valid_dataframe, scales_dataframe):
        self.valid_dataframe = valid_dataframe
        self.scales_dataframe = scales_dataframe
        self.apply_inverse = False
        if set(["y_mean","y_std"]) <= set(valid_dataframe.columns):
            self.apply_inverse = True

    def _evaluate(self, predictions):
        valid_dataframe = self.valid_dataframe.copy()
        if not self.apply_inverse:  
            valid_dataframe["ypred"] = predictions
            valid_dataframe["sq_error"] = valid_dataframe.eval("(y-ypred)**2")
        else:
            valid_dataframe["ypred"] = valid_dataframe["y_std"]*predictions + valid_dataframe["y_mean"]
            valid_dataframe["sq_error"] = valid_dataframe.eval("(y_raw-ypred)**2")
        mse = valid_dataframe.groupby("sku")["sq_error"].mean().reset_index(name="mse")
        mrg = pd.merge(mse, self.scales_dataframe, how="inner", on="sku")
        return mrg.eval("sqrt(mse)/scale").mean()

    def evaluate(self, ypred, dtrain):
        metric = self._evaluate(ypred)
        return "rmsse", metric, False

In [7]:
def ranked_probability_score(y_true, y_pred):
    """
    Input
        y_true: np.array of shape 30. 
        y_pred: np.array of shape 30. 
    """
    return ((y_true.cumsum(axis=1) - y_pred.cumsum(axis=1))**2).sum(axis=1).mean()

def scoring_function(y_true, y_pred):
    """
    Input
        y_true: List of Ints of shape Nx1. Contain the target_stock
        y_pred: List of float of shape Nx30. Contain the prob for each day
    """
    y_true_one_hot = np.zeros_like(y_pred, dtype=np.float64)
    y_true_one_hot[range(len(y_true)), y_true-1] = 1
    return ranked_probability_score(y_true_one_hot, y_pred)

***
## loading data for assessment

In [8]:
scales = pd.read_csv(f"{input_path}/scales.csv")
skus_for_assess = pd.read_csv(f"{input_path}/skus_for_assess.csv")

dataset = (
    pd.read_parquet(f"{input_path}/train.parquet")
    .query("sku in @skus_for_assess.sku")
    .rename({"date":"ds", "sold_quantity":"y"}, axis=1,)
    .reset_index(drop=True)
)
dataset

,ds,sku,y,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active,available,item_domain_id,item_id,site_id,product_id,product_family_id
0,2021-02-01,1,0,135.899994,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
1,2021-02-02,1,0,135.899994,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
2,2021-02-03,1,0,135.899994,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
3,2021-02-04,1,0,135.899994,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
4,2021-02-05,1,0,135.899994,REA,premium,fulfillment,free_shipping,0.0,1,MLB-SURFBOARD_RACKS,300279,MLB,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29179486,2021-03-27,660915,0,79.989998,MEX,classic,fulfillment,paid_shipping,0.0,1,MLM-SURGICAL_MASKS,423179,MLM,None,None
29179487,2021-03-28,660915,0,79.989998,MEX,classic,fulfillment,paid_shipping,0.0,1,MLM-SURGICAL_MASKS,423179,MLM,None,None
29179488,2021-03-29,660915,0,79.989998,MEX,classic,fulfillment,paid_shipping,0.0,1,MLM-SURGICAL_MASKS,423179,MLM,None,None
29179489,2021-03-30,660915,0,99.989998,MEX,classic,fulfillment,paid_shipping,0.0,1,MLM-SURGICAL_MASKS,423179,MLM,None,None


In [9]:
dataset.groupby("site_id")["sku"].nunique()

site_id
MLA     43314
MLB    252475
MLM    201008
Name: sku, dtype: int64

***
## model config

In [10]:
categorical_features = {
    "sku": ("y", ce.CatBoostEncoder, None),
    "currency": "default",
    "listing_type": "default",
    "shipping_logistic_type": "default",
    "shipping_payment": "default",
    "item_domain_id": "default",
    "item_id": ("y", ce.CatBoostEncoder, None),
    "site_id":"default",
    "product_id": ("y", ce.CatBoostEncoder, None),
    "product_family_id": ("y", ce.CatBoostEncoder, None),
}

exclude_features = ["fold","sold_quantity",]

model_params = {
    'objective':'tweedie',
    'tweedie_variance_power':1.6,
    'metric':'rmse',
    'num_iterations':1000,
    'max_bin': 63,
    'bin_construct_sample_cnt':20000000,
    'num_leaves': 2**8-1,
    'min_data_in_leaf': 2**8-1,
    'learning_rate': 0.02,
    'max_delta_step':1.,
    'feature_fraction':0.8,
    'bagging_fraction':0.8,
    'bagging_freq':1,
    'lambda_l2':0.1,
    'boost_from_average': False,
    'force_row_wise': True,
    'deterministic':True,
    'seed':42,
    #'device_type': 'gpu',
    #'num_gpu' : 1,
}
time_features = [
    "week_day",
    "week_day_cos",
    "week_day_sin",
    "month_progress",
]
model_kwargs = {
    "model_params":model_params,
    "time_features":time_features,
    "exclude_features":exclude_features,
    "categorical_features":categorical_features,
    "ts_uid_columns":["sku",],
}

In [11]:
dataset.loc[:,categorical_features.keys()].isna().sum(axis=0) / len(dataset)

sku                       0.000000
currency                  0.000000
listing_type              0.000000
shipping_logistic_type    0.000000
shipping_payment          0.000000
item_domain_id            0.000000
item_id                   0.000000
site_id                   0.000000
product_id                0.963178
product_family_id         0.883613
dtype: float64

In [12]:
dataset.loc[:,categorical_features.keys()].nunique()

sku                       496797
currency                       4
listing_type                   2
shipping_logistic_type         3
shipping_payment               2
item_domain_id              8156
item_id                   392139
site_id                        3
product_id                 11756
product_family_id          24176
dtype: int64

***
## model training: assessment

In [13]:
models_by_site = list()
sites = ["MLA","MLB","MLM"]

for site in sites:
    print("-"*75)
    print(f"Preparing model for site: {site}")

    tic = time.time()
    train_data = dataset.query("site_id == @site").reset_index(drop=True)
    valid_idx = train_data.query("ds > '2021-03-01'").index

    model = LightGBMForecaster(**model_kwargs)
    model.prepare_features(train_data, valid_idx)
    model.train_features = reduce_mem_usage(model.train_features, verbose=True)
    model.valid_features = reduce_mem_usage(model.valid_features, verbose=True)
    gc.collect()
    tac = time.time()
    print(f"Elapsed time for processing features: {(tac-tic)/60} min.")

    tic = time.time()
    evaluator = RMSSE(train_data.loc[valid_idx, ["sku","y"]], scales)
    model.tune(fit_kwargs={"verbose_eval":50, "feval":evaluator.evaluate})
    models_by_site.append(model)
    tac = time.time()
    print(f"Elapsed time for training the model: {(tac-tic)/60} min.")

---------------------------------------------------------------------------
Preparing model for site: MLA
Mem. usage decreased to 81.74 Mb (52.4% reduction)
Mem. usage decreased to 85.51 Mb (52.4% reduction)
Elapsed time for processing features: 0.2305383841196696 min.


[I 2021-08-18 17:57:22,223] A new study created in memory with name: no-name-5f229773-8522-452b-9b4b-38d648f955f1
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Info] Total Bins 2085
[LightGBM] [Info] Number of data points in the train set: 1242194, number of used features: 17
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 30 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's rmse: 5.76205	valid_0's rmsse: 0.849528
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGB

feature_fraction, val_score: 5.021351:  14%|#4        | 1/7 [01:53<11:23, 113.94s/it]

[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Info] Total Bins 2085
[LightGBM] [Info] Number of data points in the train set: 1242194, number of used features: 17
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=255
Training until validation scores don't improve for 30 rounds
[50]	valid_0's rmse: 5.73065	valid_0's rmsse: 0.818674
[100]	valid_0's rmse: 5.54188	valid_0's rmsse: 0.774576
[150]	valid_0's rmse: 5.31595	valid_0's rmsse: 0.769542
Early stopping, best iteration is:
[169]	valid_0's rmse: 5.23647	valid_0's rmsse: 0.76943


...

[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=25 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=25 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 5762054, number of used features: 17
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=25 will be ignored. Current value: min_data_in_leaf=255
Training until validation scores don't improve for 30 rounds
[50]	valid_0's rmse: 11.7361	valid_0's rmsse: 0.873308
[100]	valid_0's rmse: 11.6156	valid_0's rmsse: 0.796526
[150]	valid_0's rmse: 11.4228	valid_0's rmsse: 0.778654
[200]	valid_0's rmse: 11.0986	valid_0's rmsse: 0.773175
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	valid_0's rmse: 10.6868	valid_0's rmsse: 0.771914
Early stopping, best iteration is:
[262]	valid_0's rmse: 10.5932	valid_0's rmsse: 0.

min_data_in_leaf, val_score: 10.593184:  80%|########  | 4/5 [28:18<07:04, 424.41s/it]

[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=255
[LightGBM] [Info] Total Bins 2569
[LightGBM] [Info] Number of data points in the train set: 5762054, number of used features: 17
[LightGBM] [Warning] min_data_in_leaf is set=255, min_child_samples=50 will be ignored. Current value: min_data_in_leaf=255
Training until validation scores don't improve for 30 rounds
[50]	valid_0's rmse: 11.7361	valid_0's rmsse: 0.873308
[100]	valid_0's rmse: 11.6156	valid_0's rmsse: 0.796526
[150]	valid_0's rmse: 11.4228	valid_0's rmsse: 0.778654
[200]	valid_0's rmse: 11.0986	valid_0's rmsse: 0.773175
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	valid_0's rmse: 10.6868	valid_0's rmsse: 0.771914
Early stopping, best iteration is:
[262]	valid_0's rmse: 10.5932	valid_0's rmsse: 0.

min_data_in_leaf, val_score: 10.593184: 100%|##########| 5/5 [35:13<00:00, 422.79s/it]


Elapsed time for training the model: 335.8545511166255 min.


In [28]:
for site,model in zip(sites,models_by_site):
    print("-"*90)
    print(site)
    print("Metrics:", model.model.model.best_score["valid_0"])
    print("Best iteration:", model.model.model.best_iteration)
    print("config", model.model.model.params)
    
    with open(f"../config/lgbm-m1-{site}.yml", "w") as file:
        yaml.dump(model.model.model.params, file, default_flow_style=False)
        file.close()

------------------------------------------------------------------------------------------
MLA
Metrics: OrderedDict([('rmse', 5.020574388633523), ('rmsse', 0.7699177907118763)])
Best iteration: 355
config {'boosting_type': 'gbrt', 'objective': 'tweedie', 'num_leaves': 225, 'min_data_in_leaf': 255, 'learning_rate': 0.02, 'feature_fraction': 0.4, 'tweedie_variance_power': 1.6, 'metric': 'rmse', 'max_bin': 63, 'bin_construct_sample_cnt': 20000000, 'max_delta_step': 1.0, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'lambda_l2': 0.1, 'boost_from_average': False, 'force_row_wise': True, 'deterministic': True, 'seed': 42, 'feature_pre_filter': False, 'lambda_l1': 0.0, 'min_child_samples': 20, 'num_iterations': 1000, 'early_stopping_round': 30, 'categorical_column': [8, 9, 10, 11, 12, 14]}
------------------------------------------------------------------------------------------
MLB
Metrics: OrderedDict([('rmse', 7.726003926865675), ('rmsse', 0.8138287771178311)])
Best iteration: 584
config {'b

***